In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
from imblearn.combine import SMOTETomek
warnings.filterwarnings('ignore')
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Experiment 1: Train Logistic Regression Classifier

In [136]:

log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



Experiment 2: Train Random Forest Classifier

In [137]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



Experiment 3: Train XGBoost

In [138]:

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [139]:
smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [140]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



Track Experiments Using MLFlow

In [142]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [143]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [144]:
mlflow.set_experiment("Anomaly Detection 3")
mlflow.set_tracking_uri("http://127.0.0.1:5000")


for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/09/11 16:56:25 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection 3' does not exist. Creating a new experiment.
2024/09/11 16:56:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/11 16:56:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/620109767870584949/runs/745395f311ed4848b81ccc61b33fa539.
2024/09/11 16:56:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/620109767870584949.
2024/09/11 16:56:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/11 16:56:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/

Register the Model

In [149]:
model = input('Please type model-name')
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model)

Successfully registered model 'smote'.
2024/09/11 17:01:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: smote, version 1
Created version '1' of model 'smote'.
2024/09/11 17:01:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/620109767870584949/runs/de4fc2fff35241f7b5c0f621736a2d73.
2024/09/11 17:01:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/620109767870584949.


Loading the model


In [154]:
model_version = 1
model_name = 'random_forest'
# model_name = 'smote'

# model_uri = "models:/XGB-Smote/1"

# model_uri = f"models:/{model_name}/{model_version}"
# model_uri = f"models:/{model_name}@challenger"
model_uri = f"models:/{model_name}@champion"
# loaded_model = mlflow.xgboost.load_model(model_uri)
loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Transition the Model to Production

In [103]:
model_version = 2
model_name = 'random_forest'
# model_uri = "models:/XGB-Smote/1"

model_uri = f"models:/{model_name}/{model_version}"
# model_uri = f"models:/{model_name}@/challenger"
loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [75]:
current_model_uri = model_uri
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)


Registered model 'anomaly-detection-prod' already exists. Creating a new version of this model...
Copied version '3' of model 'XGB-Smote' to version '4' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1726038895554, current_stage='None', description='', last_updated_timestamp=1726038895554, name='anomaly-detection-prod', run_id='5a8c43fe2d164f9bacd59aac52ca0d9a', run_link='', source='models:/XGB-Smote/3', status='READY', status_message='', tags={}, user_id='', version='4'>

Testing on prodection model 


In [109]:
model_version = 3
model_name = 'anomaly-detection-prod'
# model_name = 'XGB-Smote'
# model_uri = "models:/XGB-Smote/1"

# model_uri = f"models:/{model_name}/{model_version}"
model_uri = f"models:/{model_name}@challenger"
# model_uri = f"models:/{model_name}@champion"
# loaded_model = mlflow.sklearn.load_model(model_uri)
loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [4]:
print(X_test)

[[ 0.12763692  0.53885434 -0.67754938 ... -0.17119613 -1.04870039
   0.1959492 ]
 [-2.10542071 -1.44658591 -3.8930874  ...  0.74058273 -3.14736801
   1.16957702]
 [-0.28933147 -0.48475775  0.04406009 ...  0.18182712  0.35313638
  -0.00841763]
 ...
 [-0.2028541  -0.0583418  -0.53918182 ...  0.04866874 -0.52587706
   0.16061758]
 [-2.51465752 -1.75645549 -4.59169829 ...  0.89256429 -3.68030212
   1.37994004]
 [-0.702736   -0.66487094 -0.93080234 ...  0.29814829 -0.55038561
   0.28270309]]
